In [1]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
                .master("spark://192.168.2.5:7077")\
                .appName("Sentiment Analysis")\
                .config("spark.cores.max", 8)\
                .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/14 09:05:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from textblob import TextBlob
import time

# Define a user-defined function to perform sentiment analysis on a text

# Start Timer
tic = time.perf_counter()

def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return 'positive'
    elif sentiment < 0:
        return 'negative'
    else:
        return 'neutral'

path = "hdfs://192.168.2.5:9000/user/ubuntu/RC_2010-03.json"
# Register the UDF with PySpark
get_sentiment_udf = udf(get_sentiment, StringType())

# Load the Reddit comments dataset into a PySpark DataFrame
df = spark_session.read.json(path)

# Add a new column to the DataFrame with the sentiment of each comment
df = df.withColumn("sentiment", get_sentiment_udf("body"))

# Group the comments by subreddit and sentiment, and count the number of comments
result = df.groupBy("subreddit", "sentiment").count()

# Show the top 10 subreddits by number of positive comments
result.filter(result.sentiment == "positive").orderBy(result["count"].desc()).show(10)

toc = time.perf_counter()

print(f"The application ran for: {toc-tic:0.4f} seconds")

spark_session.stop()



In [4]:
spark_session.stop()

In [ ]:
# Testing